# Rekurentní neuronové sítě

V předchozím modulu jsme se zabývali bohatými sémantickými reprezentacemi textu. Architektura, kterou jsme používali, zachycuje agregovaný význam slov ve větě, ale nezohledňuje **pořadí** slov, protože operace agregace, která následuje po vnoření, tuto informaci z původního textu odstraňuje. Protože tyto modely nejsou schopny reprezentovat pořadí slov, nemohou řešit složitější nebo nejednoznačné úkoly, jako je generování textu nebo odpovídání na otázky.

Abychom zachytili význam textové sekvence, použijeme architekturu neuronové sítě nazývanou **rekurentní neuronová síť** (RNN). Při použití RNN procházíme větou sítí po jednom tokenu a síť produkuje určitý **stav**, který poté předáváme síti spolu s dalším tokenem.

![Obrázek ukazující příklad generování rekurentní neuronové sítě.](../../../../../translated_images/cs/rnn.27f5c29c53d727b5.webp)

Při dané vstupní sekvenci tokenů $X_0,\dots,X_n$ RNN vytváří sekvenci bloků neuronové sítě a trénuje tuto sekvenci end-to-end pomocí zpětné propagace. Každý blok sítě přijímá dvojici $(X_i,S_i)$ jako vstup a produkuje $S_{i+1}$ jako výsledek. Konečný stav $S_n$ nebo výstup $Y_n$ se předává do lineárního klasifikátoru, aby se vytvořil výsledek. Všechny bloky sítě sdílejí stejné váhy a jsou trénovány end-to-end jedním průchodem zpětné propagace.

> Obrázek výše ukazuje rekurentní neuronovou síť v rozvinuté podobě (vlevo) a v kompaktnější rekurentní reprezentaci (vpravo). Je důležité si uvědomit, že všechny RNN buňky mají stejné **sdílené váhy**.

Protože stavové vektory $S_0,\dots,S_n$ procházejí sítí, RNN je schopna se naučit sekvenční závislosti mezi slovy. Například když se někde v sekvenci objeví slovo *not*, může se naučit negovat určité prvky ve stavovém vektoru.

Uvnitř každé RNN buňky jsou dvě matice vah: $W_H$ a $W_I$, a bias $b$. Při každém kroku RNN, při daném vstupu $X_i$ a vstupním stavu $S_i$, se výstupní stav vypočítá jako $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, kde $f$ je aktivační funkce (často $\tanh$).

> U problémů, jako je generování textu (které pokryjeme v další jednotce) nebo strojový překlad, chceme také získat nějakou výstupní hodnotu při každém kroku RNN. V tomto případě existuje další matice $W_O$ a výstup se vypočítá jako $Y_i=f(W_O\times S_i+b_O)$.

Podívejme se, jak nám rekurentní neuronové sítě mohou pomoci klasifikovat naši datovou sadu zpráv.

> Pro sandboxové prostředí je třeba spustit následující buňku, abychom zajistili, že požadovaná knihovna je nainstalována a data jsou předem načtena. Pokud pracujete lokálně, můžete tuto buňku přeskočit.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Při trénování velkých modelů může být problémem alokace paměti GPU. Také může být potřeba experimentovat s různými velikostmi minibatchů, aby se data vešla do paměti GPU a zároveň bylo trénování dostatečně rychlé. Pokud tento kód spouštíte na svém vlastním GPU stroji, můžete experimentovat s úpravou velikosti minibatchů pro zrychlení trénování.

> **Note**: Je známo, že některé verze ovladačů NVidia neuvolňují paměť po trénování modelu. V tomto notebooku spouštíme několik příkladů, což může v určitých konfiguracích způsobit vyčerpání paměti, zejména pokud provádíte vlastní experimenty v rámci stejného notebooku. Pokud narazíte na podivné chyby při zahájení trénování modelu, může být vhodné restartovat kernel notebooku.


In [3]:
batch_size = 16
embed_size = 64

## Jednoduchý RNN klasifikátor

V případě jednoduchého RNN je každá rekurentní jednotka jednoduchou lineární sítí, která přijímá vstupní vektor a stavový vektor a vytváří nový stavový vektor. V Kerasu to lze reprezentovat pomocí vrstvy `SimpleRNN`.

I když můžeme předávat tokeny zakódované metodou one-hot přímo do vrstvy RNN, není to dobrý nápad kvůli jejich vysoké dimenzionalitě. Proto použijeme vrstvu embedding ke snížení dimenzionality slovních vektorů, následovanou vrstvou RNN a nakonec klasifikátorem `Dense`.

> **Note**: V případech, kdy dimenzionalita není tak vysoká, například při použití tokenizace na úrovni znaků, může být smysluplné předávat tokeny zakódované metodou one-hot přímo do buňky RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Poznámka:** Zde používáme netrénovanou vrstvu vkládání pro zjednodušení, ale pro lepší výsledky můžeme použít předtrénovanou vrstvu vkládání pomocí Word2Vec, jak bylo popsáno v předchozí jednotce. Bylo by dobrým cvičením upravit tento kód tak, aby fungoval s předtrénovanými vkládáními.

Nyní pojďme natrénovat naši RNN. RNN jsou obecně poměrně obtížné trénovat, protože jakmile jsou buňky RNN rozvinuty podél délky sekvence, výsledný počet vrstev zapojených do zpětné propagace je velmi vysoký. Proto je potřeba zvolit menší rychlost učení a trénovat síť na větším datasetu, aby se dosáhlo dobrých výsledků. To může trvat poměrně dlouho, takže je preferováno použití GPU.

Abychom proces urychlili, budeme model RNN trénovat pouze na titulcích zpráv a vynecháme popis. Můžete zkusit trénovat i s popisem a zjistit, zda se vám podaří model natrénovat.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Poznámka** že přesnost bude pravděpodobně nižší, protože trénujeme pouze na titulcích zpráv.


## Znovu se podívejme na sekvence proměnných

Pamatujte, že vrstva `TextVectorization` automaticky doplní sekvence proměnné délky v minibatchi pomocí padovacích tokenů. Ukazuje se, že tyto tokeny se také účastní trénování a mohou komplikovat konvergenci modelu.

Existuje několik přístupů, které můžeme použít ke snížení množství paddingu. Jedním z nich je přeřazení datasetu podle délky sekvence a seskupení všech sekvencí podle velikosti. To lze provést pomocí funkce `tf.data.experimental.bucket_by_sequence_length` (viz [dokumentace](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Dalším přístupem je použití **maskování**. V Keras některé vrstvy podporují dodatečný vstup, který ukazuje, které tokeny by měly být zohledněny při trénování. Abychom do našeho modelu začlenili maskování, můžeme buď přidat samostatnou vrstvu `Masking` ([dokumentace](https://keras.io/api/layers/core_layers/masking/)), nebo můžeme specifikovat parametr `mask_zero=True` u naší vrstvy `Embedding`.

> **Note**: Toto trénování zabere přibližně 5 minut na dokončení jedné epochy na celém datasetu. Pokud vám dojde trpělivost, můžete trénování kdykoli přerušit. Další možností je omezit množství dat použitých pro trénování přidáním klauzule `.take(...)` po datasetech `ds_train` a `ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Nyní, když používáme maskování, můžeme model trénovat na celém datasetu titulků a popisů.

> **Note**: Všimli jste si, že jsme používali vektorizér natrénovaný na titulcích zpráv, a ne na celém textu článku? To může potenciálně způsobit, že některé tokeny budou ignorovány, takže je lepší vektorizér přeškolit. Nicméně, efekt by mohl být velmi malý, takže pro zjednodušení zůstaneme u předchozího předtrénovaného vektorizéru.


## LSTM: Dlouhá krátkodobá paměť

Jedním z hlavních problémů RNN je **mizení gradientů**. RNN mohou být poměrně dlouhé a mohou mít potíže s propagací gradientů zpět až k první vrstvě sítě během zpětného šíření. Když k tomu dojde, síť se nedokáže naučit vztahy mezi vzdálenými tokeny. Jedním ze způsobů, jak tento problém obejít, je zavedení **explicitního řízení stavu** pomocí **bran**. Dvě nejběžnější architektury, které zavádějí brány, jsou **dlouhá krátkodobá paměť** (LSTM) a **gated relay unit** (GRU). Zde se budeme věnovat LSTM.

![Obrázek ukazující příklad buňky dlouhé krátkodobé paměti](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM síť je organizována podobně jako RNN, ale existují dva stavy, které se předávají z vrstvy do vrstvy: aktuální stav $c$ a skrytý vektor $h$. V každé jednotce se skrytý vektor $h_{t-1}$ kombinuje se vstupem $x_t$ a společně určují, co se stane se stavem $c_t$ a výstupem $h_{t}$ prostřednictvím **bran**. Každá brána má sigmoidní aktivaci (výstup v rozmezí $[0,1]$), kterou si můžeme představit jako bitovou masku při násobení stavovým vektorem. LSTM mají následující brány (zleva doprava na obrázku výše):
* **zapomínací brána**, která určuje, které složky vektoru $c_{t-1}$ je třeba zapomenout a které propustit dál.
* **vstupní brána**, která určuje, kolik informací ze vstupního vektoru a předchozího skrytého vektoru by mělo být začleněno do stavového vektoru.
* **výstupní brána**, která vezme nový stavový vektor a rozhodne, které jeho složky budou použity k vytvoření nového skrytého vektoru $h_t$.

Složky stavu $c$ si můžeme představit jako příznaky, které lze zapínat a vypínat. Například když v sekvenci narazíme na jméno *Alice*, odhadneme, že se jedná o ženu, a nastavíme příznak ve stavu, který říká, že máme v větě ženské podstatné jméno. Když dále narazíme na slova *a Tom*, nastavíme příznak, který říká, že máme množné číslo podstatného jména. Manipulací se stavem tak můžeme sledovat gramatické vlastnosti věty.

> **Note**: Zde je skvělý zdroj pro pochopení vnitřní struktury LSTM: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) od Christophera Olaha.

I když může vnitřní struktura LSTM buňky vypadat složitě, Keras tuto implementaci skrývá uvnitř vrstvy `LSTM`, takže jediná věc, kterou musíme udělat v příkladu výše, je nahradit rekurentní vrstvu:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## Obousměrné a vícevrstvé RNN

V našich dosavadních příkladech pracují rekurentní sítě od začátku sekvence až do jejího konce. To nám připadá přirozené, protože to odpovídá směru, kterým čteme nebo posloucháme řeč. Nicméně v situacích, které vyžadují náhodný přístup k vstupní sekvenci, dává větší smysl provádět rekurentní výpočty v obou směrech. RNN, které umožňují výpočty v obou směrech, se nazývají **obousměrné** RNN a lze je vytvořit obalením rekurentní vrstvy speciální vrstvou `Bidirectional`.

> **Note**: Vrstva `Bidirectional` vytvoří dvě kopie vrstvy uvnitř sebe a nastaví vlastnost `go_backwards` jedné z těchto kopií na hodnotu `True`, což způsobí, že bude procházet sekvenci opačným směrem.

Rekurentní sítě, ať už jednosměrné nebo obousměrné, zachycují vzory v rámci sekvence a ukládají je do stavových vektorů nebo je vracejí jako výstup. Stejně jako u konvolučních sítí můžeme vytvořit další rekurentní vrstvu, která následuje po první, aby zachytila vzory na vyšší úrovni, vytvořené z nižších úrovní vzorů extrahovaných první vrstvou. To nás přivádí k pojmu **vícevrstvé RNN**, které se skládají ze dvou nebo více rekurentních sítí, kde výstup předchozí vrstvy je předán jako vstup další vrstvě.

![Obrázek znázorňující vícevrstvou dlouhodobou krátkodobou paměťovou RNN](../../../../../translated_images/cs/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Obrázek z [tohoto skvělého příspěvku](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) od Fernanda Lópeze.*

Keras usnadňuje konstrukci těchto sítí, protože stačí přidat více rekurentních vrstev do modelu. U všech vrstev kromě poslední je třeba specifikovat parametr `return_sequences=True`, protože potřebujeme, aby vrstva vracela všechny mezistavy, a nejen konečný stav rekurentního výpočtu.

Postavme dvouvrstvou obousměrnou LSTM pro náš klasifikační problém.

> **Note** tento kód opět trvá poměrně dlouho, než se dokončí, ale poskytuje nám nejvyšší přesnost, jakou jsme dosud viděli. Možná tedy stojí za to počkat a podívat se na výsledek.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN pro jiné úkoly

Doposud jsme se zaměřovali na používání RNN k klasifikaci sekvencí textu. Ale zvládnou mnohem více úkolů, jako je generování textu a strojový překlad — těmto úkolům se budeme věnovat v další jednotce.



---

**Upozornění**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o co největší přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho původním jazyce. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
